In [1]:
from IPython.display import clear_output
!pip install ultralytics
clear_output()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
from PIL import Image
import csv
import os
import random
import torch

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

In [3]:
def filter_invalid_boxes(boxes, scores, labels):
    filtered_boxes, filtered_scores, filtered_labels = [], [], []
    for b, s, l in zip(boxes, scores, labels):
        if abs(b[2] - b[0]) > 1e-6 and abs(b[3] - b[1]) > 1e-6:
            filtered_boxes.append(b)
            filtered_scores.append(s)
            filtered_labels.append(l)
    return filtered_boxes, filtered_scores, filtered_labels
    
def run_inference(models, image_sizes, test_images_path, agnostic_nms=False, iou=0.4, key="default"):
    image_paths = [p for p in Path(test_images_path).glob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]
    predictions = {}

    for model_idx, model in enumerate(models):
        model.eval()
        model.training = False
        predictions[model_idx] = {}
        os.makedirs(f"{key}/{model_idx}", exist_ok=True)
        
        for size in image_sizes:
            predictions[model_idx][size] = {}
            pred = []
            for img_path in image_paths:
                image_id = img_path.stem
                image = Image.open(img_path)
                img_width, img_height = image.size

                results = model.predict(source=str(img_path), conf=conf,iou=iou, max_det=600, agnostic_nms=agnostic_nms, augment=True, imgsz=size, verbose=False)
                boxes, scores, labels = [], [], []

                for result in results:
                    if result.boxes is None:
                        continue
                    boxes = result.boxes.xyxy.cpu().numpy().tolist()
                    scores = result.boxes.conf.cpu().numpy().tolist()
                    labels = result.boxes.cls.cpu().numpy().tolist()

                    norm_boxes = [
                        [x1 / img_width, y1 / img_height, x2 / img_width, y2 / img_height]
                        for x1, y1, x2, y2 in boxes
                    ]
                    norm_boxes, scores, labels = filter_invalid_boxes(norm_boxes, scores, labels)

                predictions[model_idx][size][image_id] = {
                    "boxes": norm_boxes,
                    "scores": scores,
                    "labels": labels
                }
                
                if boxes:
                    prediction_string = " ".join(
                        f"{int(lbl)} {score:.6f} {(b[0]+b[2])/2:.6f} {(b[1]+b[3])/2:.6f} {(b[2]-b[0]):.6f} {(b[3]-b[1]):.6f}"
                        for b, score, lbl in zip(norm_boxes, scores, labels)
                    )
                else:
                    prediction_string = "no boxes"

                pred.append({
                    "image_id": image_id,
                    "prediction_string": prediction_string
                })

            # Save CSV per model and size
            df = pd.DataFrame(pred)
            csv_path = f"{key}/{model_idx}/{size}.csv"
            df.to_csv(csv_path, index=False, quoting=csv.QUOTE_MINIMAL)
            print(f"[saved] {csv_path}")
            print(df.shape)
            print(df.head(10))

    return predictions



In [ ]:

# === Config ===

model_paths = [
    "/kaggle/input/falcon-multiclass-yolo-train/runs/detect/train/weights/best.pt",
    "/kaggle/input/falcon-multiclass-yolo-train/runs/detect/train2/weights/best.pt",
    "/kaggle/input/falcon-multiclass-yolo-train/runs/detect/train22/weights/best.pt",
    "/kaggle/input/falcon-multiclass-yolo-train/runs/detect/train222/weights/best.pt",
    "/kaggle/input/falcon-multiclass-yolo-train/runs/detect/train2222/weights/best.pt",
]
test_images_path = "/kaggle/input/multi-class-object-detection-challenge/testImages/images"
conf = 0
image_sizes = [640, 800, 1056, 1376, 1600, 1920]


models = [YOLO(path) for path in model_paths]

tasks = {"default": (False, 0.4)}

for key, values in tasks.items():
    agnms, iou = values
    run_inference(models, image_sizes, test_images_path, iou=iou, agnostic_nms=agnms, key=key)





[saved] default/0/640.csv
(229, 2)
   image_id                                  prediction_string
0  IMG_8712  0 0.988986 0.460025 0.488498 0.260656 0.247856...
1  IMG_8949  0 0.992652 0.521739 0.514556 0.227738 0.252245...
2  IMG_8778  0 0.987490 0.426535 0.736258 0.522379 0.450927...
3  IMG_8723  0 0.977244 0.280729 0.509077 0.189260 0.302615...
4  IMG_8771  1 0.977200 0.704487 0.497719 0.182001 0.209258...
5  IMG_9164  0 0.988026 0.668358 0.495484 0.628854 0.350950...
6  IMG_9946  0 0.950534 0.274385 0.144704 0.235058 0.289408...
7  IMG_9953  1 0.975076 0.653702 0.620132 0.118390 0.179962...
8  IMG_8796  1 0.974870 0.348341 0.531711 0.193522 0.237166...
9  IMG_9951  1 0.392412 0.638941 0.559883 0.073106 0.108611...
